In [0]:
import random
random.seed(109)

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegressionCV
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

In [0]:
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import utils

In [0]:
#Read in the cleaned dataset
df_url = 'https://raw.githubusercontent.com/xenaka/cs109_final_project/master/Datasets/df_merge_CPS.csv'
df_final = pd.read_csv(df_url)

In [0]:
df_final.loc[df_final['DAY_OF_WEEK'] == 'Sunday', 'DAY_OF_WEEK_NUM'] = 0
df_final.loc[df_final['DAY_OF_WEEK'] == 'Monday', 'DAY_OF_WEEK_NUM'] = 1
df_final.loc[df_final['DAY_OF_WEEK'] == 'Tuesday', 'DAY_OF_WEEK_NUM'] = 2
df_final.loc[df_final['DAY_OF_WEEK'] == 'Wednesday', 'DAY_OF_WEEK_NUM'] = 3
df_final.loc[df_final['DAY_OF_WEEK'] == 'Thursday', 'DAY_OF_WEEK_NUM'] = 4
df_final.loc[df_final['DAY_OF_WEEK'] == 'Friday', 'DAY_OF_WEEK_NUM'] = 5
df_final.loc[df_final['DAY_OF_WEEK'] == 'Saturday', 'DAY_OF_WEEK_NUM'] = 6

df_final.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,OCCURED_TIME,Crime_Type,Crime_Type_Cat,ST_NAME_1st,ST_NAME_2nd,ST_NAME_SUF,ST_comb,ST_12,ST_1SUF,ZIPCODE,AV_BLDG,AV_TOTAL,GROSS_TAX,LAND_SF,GROSS_AREA,LIVING_AREA,NUM_FLOORS,PTYPE_A,PTYPE_C,PTYPE_EO,PTYPE_EP,PTYPE_I,PTYPE_MU,PTYPE_R,YR_BUILT_m,YR_REMOD_m,SHOOTING_DUMMY,Land area,Population Density,Median household income,Total Population,Younger Population,Male Population,Young_prop,Dist_to_Nearest_Light,Lights_within_50m,Lights_within_100m,DAY_OF_WEEK_NUM
0,I192078623,3802,Motor Vehicle Accident Response,M/V ACCIDENT - PROPERTY DAMAGE,C11,400,NaN,2019-09-28 22:40:00,2019,9,Saturday,22,Part Three,MONTAGUE ST,42.286065,-71.070010,"(42.28606484, -71.07001038)",1900-01-01 22:40:00,4.0,M/V Accident,MONTAGUE,ST,ST,MONTAGUE ST,MONTAGUE ST,MONTAGUE ST,2124,542950.000000,707000.000000,745177.750000,5070.000000,4008.500000,2602.500000,2.125000,0.0,0.0,0.0,0.0,0.0,0.0,4.0,-15.750000,14.000000,0.0,3.00,15913.0,48841.0,47783.0,13866.0,21824.0,0.290187,7.813019,4,13,6.0
1,I192078599,1402,Vandalism,VANDALISM,B2,238,NaN,2019-09-28 23:21:00,2019,9,Saturday,23,Part Two,WENDOVER ST,42.318827,-71.066054,"(42.31882655, -71.06605368)",1900-01-01 23:21:00,3.0,Vandalism,WENDOVER,ST,ST,WENDOVER ST,WENDOVER ST,WENDOVER ST,2125,288082.500000,413481.000000,476224.800000,3479.850000,4108.300000,2827.650000,2.500000,2.0,0.0,0.0,0.0,0.0,1.0,17.0,-15.944444,-1.000000,0.0,2.13,15611.0,45130.0,33295.0,8835.0,16399.0,0.265355,6.694126,10,48,6.0
2,I192078586,3802,Motor Vehicle Accident Response,M/V ACCIDENT - PROPERTY DAMAGE,E18,485,NaN,2019-09-28 23:33:00,2019,9,Saturday,23,Part Three,MATTAKEESET ST,42.264837,-71.099668,"(42.26483674, -71.09966819)",1900-01-01 23:33:00,4.0,M/V Accident,MATTAKEESET,ST,ST,MATTAKEESET ST,MATTAKEESET ST,MATTAKEESET ST,2136,285243.478261,426378.260870,449402.826087,5553.869565,3627.173913,2069.565217,1.956522,0.0,0.0,0.0,0.0,0.0,0.0,23.0,-3.347826,-15.000000,0.0,4.59,6207.0,58890.0,28488.0,8055.0,13324.0,0.282751,10.215599,3,5,6.0
3,I192078583,3801,Motor Vehicle Accident Response,M/V ACCIDENT - OTHER,E18,478,NaN,2019-09-28 23:26:00,2019,9,Saturday,23,Part Three,GREENFIELD RD,42.270801,-71.105812,"(42.27080089, -71.10581184)",1900-01-01 23:26:00,4.0,M/V Accident,GREENFIELD,RD,RD,GREENFIELD RD,GREENFIELD RD,GREENFIELD RD,2126,231282.448276,328715.672414,349293.810345,4681.810345,3011.189655,1668.793103,1.730769,3.0,1.0,0.0,0.0,0.0,0.0,54.0,16.057692,-0.444444,0.0,2.08,12277.0,45238.0,25562.0,7076.0,11547.0,0.276817,61.418527,0,8,6.0
4,I192078582,3831,Motor Vehicle Accident Response,M/V - LEAVING SCENE - PROPERTY DAMAGE,E13,906,NaN,2019-09-28 22:26:00,2019,9,Saturday,22,Part Three,WALDEN ST,42.325610,-71.104500,"(42.32561013, -71.10449956)",1900-01-01 22:26:00,4.0,M/V Accident,WALDEN,ST,ST,WALDEN ST,WALDEN ST,WALDEN ST,2130,253139.250000,391363.800000,349905.800000,5926.250000,3818.250000,2559.000000,2.666667,8.0,1.0,4.0,0.0,0.0,0.0,7.0,22.666667,-6.222222,0.0,3.33,10618.0,75730.0,35401.0,6177.0,16230.0,0.174487,4.517171,8,39,6.0


# Model

In [0]:
# All predictors that could potentially be used
pred_col_names = ['SHOOTING_DUMMY', 'HOUR', 'MONTH', 'DAY_OF_WEEK_NUM', 
                  'AV_BLDG', 'AV_TOTAL', 'LAND_SF', 'GROSS_TAX', 'GROSS_AREA', 'LIVING_AREA', 'NUM_FLOORS',
                  'PTYPE_A', 'PTYPE_C', 'PTYPE_EO', 'PTYPE_EP', 'PTYPE_I', 'PTYPE_MU', 'PTYPE_R', 'YR_BUILT_m', 'YR_REMOD_m', 
                  'Population Density', 'Young_prop', 'Median household income', 
                  'Dist_to_Nearest_Light', 'Lights_within_50m', 'Lights_within_100m']

In [0]:
# drop missing values
df_model = df_final.dropna(subset = pred_col_names)

In [0]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(df_model, 
                                                    df_model.Crime_Type, test_size=0.2, 
                                                    random_state = 8, 
                                                    stratify = df_model.Crime_Type)

### Baseline (Multiple Logistic Regression Model)

In [0]:
# drop 'AV_BLDG', 'Median household income','GROSS_AREA' and Lights_within_100m'
pred_col_log = ['SHOOTING_DUMMY', 'HOUR', 'MONTH', 'DAY_OF_WEEK_NUM', 
                  'AV_TOTAL', 'LAND_SF', 'GROSS_TAX', 'LIVING_AREA', 'NUM_FLOORS',
                  'PTYPE_A', 'PTYPE_C', 'PTYPE_EO', 'PTYPE_EP', 'PTYPE_I', 'PTYPE_MU', 'PTYPE_R', 'YR_BUILT_m', 'YR_REMOD_m', 
                  'Population Density', 'Young_prop', 
                  'Dist_to_Nearest_Light', 'Lights_within_50m']

In [0]:
#Standardize
continuous = ['AV_BLDG', 'AV_TOTAL', 'LAND_SF', 'GROSS_TAX', 'GROSS_AREA', 'LIVING_AREA', 'NUM_FLOORS',
              'PTYPE_A', 'PTYPE_C', 'PTYPE_EO', 'PTYPE_EP', 'PTYPE_I', 'PTYPE_MU', 'PTYPE_R', 'YR_BUILT_m', 'YR_REMOD_m', 
              'Population Density', 'Young_prop', 'Median household income', 
              'Dist_to_Nearest_Light', 'Lights_within_50m', 'Lights_within_100m']

scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_train_scaled[continuous] = scaler.fit_transform(X_train_scaled[continuous])

X_test_scaled = X_test.copy()
X_test_scaled[continuous] = scaler.transform(X_test_scaled[continuous])

In [0]:
#Turn hour, month and day of week into dummy variables
dummies = ['HOUR', 'MONTH', 'DAY_OF_WEEK_NUM']

X_train_dum = pd.get_dummies(X_train_scaled.loc[:,pred_col_log], columns=dummies, drop_first=True)
X_test_dum = pd.get_dummies(X_test_scaled.loc[:,pred_col_log], columns=dummies,drop_first=True)

In [0]:
#Fit the model
model_logcv = LogisticRegressionCV(multi_class='ovr', cv=5, max_iter=200).fit(X_train_dum, y_train)

In [0]:
print('The accuracy score of the testing data is :', accuracy_score(y_test.values, model_logcv.predict(X_test_dum)))

The accuracy score of the testing data is : 0.3394721577726218


In [0]:
print('The accuracy score of the training data is :', accuracy_score(y_train.values, model_logcv.predict(X_train_dum)))

The accuracy score of the training data is : 0.3357109840210998


### Optimized Logistic Regression Model

In [0]:
#All variables were dumped in for feature selection
X_train_log2 = pd.get_dummies(X_train_scaled.loc[:,pred_col_names], columns=dummies,drop_first=True)
X_test_log2 = pd.get_dummies(X_test_scaled.loc[:,pred_col_names], columns=dummies,drop_first=True)

In [0]:
#Fit the training data into the classifier
estimator = ExtraTreesClassifier(n_estimators = 10)
featureSelection = SelectFromModel(estimator)
featureSelection.fit(X_train_log2, y_train)

#Select the important features
selectedFeatures = featureSelection.transform(X_train_log2)
Selected_var = X_train_log2.columns[featureSelection.get_support()]

In [0]:
#Fit the logistic model with the selected variables
ovr_model=LogisticRegressionCV(multi_class = 'ovr', cv=5).fit(X_train_log2.loc[:,list(Selected_var)],y_train)
print('The accuracy in train dataset is '+"{}".format(ovr_model.score(X_train_log2.loc[:,list(Selected_var)], y_train)))
print('The accuracy in test dataset is '+"{}".format(ovr_model.score(X_test_log2.loc[:,list(Selected_var)], y_test)))

The accuracy in training dataset is 0.31297073405055603
The accuracy in testing dataset is 0.3130800464037123


### Decision tree

In [0]:
#Decision Tree 
#Function for finding the best depth and fit the tree with it 
def Find_BestTree(X_train, y_train, depths):
    cvmeans = []
    cvstds = []
    train_scores = []
    for i in depths:
        tree = DecisionTreeClassifier(max_depth=i)
        model_tree = tree.fit(X_train, y_train)

        score = cross_val_score(estimator=tree, X=X_train, y=y_train, cv=5)
        train_score = accuracy_score(y_train, model_tree.predict(X_train))        
        cvmeans.append(score.mean())
        cvstds.append(score.std())
        train_scores.append(train_score)

    best_depth = depths[cvmeans.index(sorted(cvmeans, reverse=True)[0])]

    # Fit a tree with the best depth 
    tree_best = DecisionTreeClassifier(max_depth=best_depth)
    tree_best.fit(X_train, y_train)

    return best_depth, tree_best, cvmeans, cvstds, train_scores

depths = list(range(1, 61))
best_depth, model_BestTree, cvmeans, cvstds, train_scores= Find_BestTree(X_train.loc[:,pred_col_names], y_train, depths)

print('The accuracy score in the train set of the decision tree with the best depth={} is: {}'.format(best_depth, accuracy_score(y_train.values, model_BestTree.predict(X_train.loc[:,pred_col_names]))))
print('The accuracy score in the test set of the decision tree with the best depth={} is: {}'.format(best_depth, accuracy_score(y_test.values, model_BestTree.predict(X_test.loc[:,pred_col_names]))))

In [0]:
# Plotting means and standard deviations for different depths 
plt.figure(figsize=(10, 6))
plt.fill_between(depths, np.array(cvmeans) + np.array(cvstds), np.array(cvmeans) - np.array(cvstds), alpha=0.5)
plt.ylabel("Cross Validation Accuracy")
plt.xlabel("Maximum Depth")
plt.plot(depths, train_scores, 'b-', marker='o', label = "Train set performance")
plt.plot(depths, cvmeans, 'r-', marker='o', label = "Cross-validation performance")
plt.legend()
plt.show()

### Random Forest

In [0]:
#Random Forest 
tree_depth = best_depth

n_trees = np.arange(400,500,20)
rf_train_score = []
rf_test_score = []
for n in n_trees:
    rf_model = RandomForestClassifier(n_estimators=n, max_depth=tree_depth, max_features = 'auto')
    rf_model.fit(X_train.loc[:,pred_col_names], y_train)
    train_sc = accuracy_score(y_train.values, rf_model.predict(X_train.loc[:,pred_col_names]))
    test_sc = accuracy_score(y_test.values, rf_model.predict(X_test.loc[:,pred_col_names]))
    rf_train_score.append(train_sc)
    rf_test_score.append(test_sc)
    print("Finished n={}".format(n))

print('With number of trees = {} and maximum depth = {}, the optimized test set accuracy score reaches {}'.format(n_trees[rf_test_score.index(max(rf_test_score))], best_depth, max(rf_test_score)))

### Boosting

In [0]:
#Boosting 
boost_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=15),
                                 n_estimators=100,
                                 learning_rate=0.1)
boost_model.fit(X_train.loc[:,pred_col_names], y_train)

staged_boosting_training = list(boost_model.staged_score(X_train.loc[:,pred_col_names], y_train))
staged_boosting_test = list(boost_model.staged_score(X_test.loc[:,pred_col_names], y_test))

In [0]:
plt.figure(figsize=(10, 6))

#Plot 
plt.ylabel("Accuracy Score")
plt.xlabel("Number of Iterations")
plt.plot(range(1,101),
         staged_boosting_training,
         'b-', label = "Train set performance")
plt.plot(range(1,101),
         staged_boosting_test,
         'r-', label = "Test set performance")
plt.legend()
plt.show()

### Neural Network

In [0]:
#Neural Network 
nn_model = models.Sequential(
  [layers.Dense(256, activation='relu', input_shape=(X_train_scaled.loc[:,pred_col_names].shape[1],)),
   layers.Dropout(0.2),
   layers.Dense(256, activation='relu'),
   layers.Dropout(0.2),
   layers.Dense(256, activation='relu'),
   layers.Dropout(0.2),
   layers.Dense(256, activation='relu'),

   layers.Dropout(0.2),

  layers.Dense(6, activation='softmax')])


adam = optimizers.Adam(lr=0.0008)
nn_model.compile(loss='sparse_categorical_crossentropy',
               optimizer= adam,
               metrics=['accuracy'])

hist = nn_model.fit(X_train_scaled.loc[:,pred_col_names], y_train, shuffle=True, batch_size=64, epochs=1000, validation_split = 0.2,
                    verbose=1)

#Test set accuracy 
accuracy_score(y_test, nn_model.predict_classes(X_test_scaled.loc[:,pred_col_names]))

In [0]:
#Plot 
val_acc = hist.history["val_accuracy"]
acc = hist.history["accuracy"]

plt.figure(figsize=(15,6))

epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.legend()

plt.show()

### Confusion Matrix

In [0]:
#Baseline Model Confusion Matrix
crime_labels = ['Theft', 'Robbery', 'Assault', 'Vandalism', 'M/V Accident', 'Drug Abuse Violations']

matrix_logcv = confusion_matrix(y_test.values, model_logcv.predict(X_test_dum))
plt.figure(figsize = (12,10))
sns.set(font_scale=1)
sns.heatmap(matrix_logcv,annot=True,cbar=True, xticklabels=crime_labels, yticklabels=crime_labels)
plt.xticks(rotation=30)
plt.yticks(rotation=30)
plt.ylabel('True Value')
plt.xlabel('Predicted Value');

In [0]:
#Optimized Logistic Model Confusion Matrix
matrix_log2 = confusion_matrix(y_test.values, ovr_model.predict(X_test_log2.loc[:,list(Selected_var)]))
plt.figure(figsize = (12,10))
sns.set(font_scale=1)
sns.heatmap(matrix_log2,annot=True,cbar=True, xticklabels=crime_labels, yticklabels=crime_labels)
plt.xticks(rotation=30)
plt.yticks(rotation=30)
plt.ylabel('True Value')
plt.xlabel('Predicted Value');

In [0]:
#Neural Network Confusion Matrix
matrix_NN = confusion_matrix(y_test.values, nn_model.predict_classes(X_test_scaled.loc[:,pred_col_names]))
plt.figure(figsize = (12,10))
sns.set(font_scale=1)
sns.heatmap(matrix_NN,annot=True,cbar=True, xticklabels=crime_labels, yticklabels=crime_labels)
plt.xticks(rotation=30)
plt.yticks(rotation=30)
plt.ylabel('True Value')
plt.xlabel('Predicted Value');

In [0]:
#Decision Tree Confusion Matrix
matrix_deci_tree = confusion_matrix(y_test.values, model_BestTree.predict(X_test.loc[:,pred_col_names]))
plt.figure(figsize = (12,10))
sns.set(font_scale=1)
sns.heatmap(matrix_deci_tree,annot=True,cbar=True, xticklabels=crime_labels, yticklabels=crime_labels)
plt.xticks(rotation=30)
plt.yticks(rotation=30)
plt.ylabel('True Value')
plt.xlabel('Predicted Value');

In [0]:
#Random Forest Confusion Matrix
rf_model = RandomForestClassifier(n_estimators=420, max_depth=tree_depth, max_features = 'auto')
rf_model.fit(X_train.loc[:,pred_col_names], y_train)

matrix_rf = confusion_matrix(y_test.values, rf_model.predict(X_test.loc[:,pred_col_names]))
plt.figure(figsize = (12,10))
sns.set(font_scale=1)
sns.heatmap(matrix_rf,annot=True,cbar=True, xticklabels=crime_labels, yticklabels=crime_labels)
plt.xticks(rotation=30)
plt.yticks(rotation=30)
plt.ylabel('True Value')
plt.xlabel('Predicted Value');

In [0]:
#Boosting Confusion Matrix
matrix_boost = confusion_matrix(y_test.values, boost_model.predict(X_test.loc[:,pred_col_names]))
plt.figure(figsize = (12,10))
sns.set(font_scale=1)
sns.heatmap(matrix_boost,annot=True,cbar=True, xticklabels=crime_labels, yticklabels=crime_labels)
plt.xticks(rotation=30)
plt.yticks(rotation=30)
plt.ylabel('True Value')
plt.xlabel('Predicted Value');

### Feature Importance

In [0]:
#Important Features in RF and Boosting models 
boost_predictors_imp = pd.DataFrame({'predictor': X_train.loc[:,pred_col_names].columns, 'relative importance':boost_model.feature_importances_}).sort_values(by='relative importance', ascending=False)
rf_predictors_imp = pd.DataFrame({'predictor': X_train.loc[:,pred_col_names].columns, 'relative importance':rf_model.feature_importances_}).sort_values(by='relative importance', ascending=False)

In [0]:
#Plot
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(45,20))
fig.subplots_adjust(wspace = 0.4)
y_pos = np.arange(len(boost_predictors_imp ))

ax[0].barh(y_pos, rf_predictors_imp['relative importance'], align='center')
ax[0].xaxis.set_tick_params(labelsize=25)
ax[0].set_yticks(y_pos)
ax[0].set_yticklabels(rf_predictors_imp['predictor'],fontsize=27)
ax[0].invert_yaxis()  
ax[0].set_xlabel('Relative Importance',fontsize=27)
ax[0].set_title('Random Forest',fontsize=30, fontweight="bold")

ax[1].barh(y_pos, boost_predictors_imp['relative importance'], align='center')
ax[1].xaxis.set_tick_params(labelsize=25)
ax[1].set_yticks(y_pos)
ax[1].set_yticklabels(boost_predictors_imp['predictor'],fontsize=27)
ax[1].invert_yaxis()  
ax[1].set_xlabel('Relative Importance',fontsize=27)
ax[1].set_title('Boosting',fontsize=30, fontweight="bold")

plt.show()